<h1 style="color: #5e9ca0;"><span style="color: #2b2301;">PA Cloud </span>and&nbsp;<span style="color: #2b2301;">SalesForce.com</span> integration using Watson Studio</h1>

In [ ]:
#Start by installing simple-salesforce python library
!pip install simple-salesforce
#Also install TM1Py Open source Python Library for TM1 Rest API by Cubewise Code
!pip install TM1py

In [34]:
import pandas as pd
from pandas import DataFrame
from simple_salesforce import Salesforce

In [46]:
creds = pd.read_csv(body)

In [36]:
#For SalesForce.com
#bring credentials from csv or enter below
#creds = pd.read_csv("SalesForce/upassword.csv")
#SalesForce username and password > A free developer account can be setup here: https://developer.salesforce.com/
sfuser = creds.iloc[0,4]
sfpass = creds.iloc[0,5]
#for dev environments isSandbox = 0
isSandbox = 0
#this token is generated as so https://help.salesforce.com/articleView?id=user_security_token.htm&type=5
sftoken = creds.iloc[0,6]
#instance is provided to you. easiest way to find it is to look under your profile or go to https://help.salesforce.com/articleView?id=000322728&language=en_US&type=1&mode=1
sfinstance = creds.iloc[0,7]

In [37]:
#establish SalesForce connect syntax
sf = Salesforce(username = sfuser,
              password = sfpass,
              security_token = sftoken,
              instance_url = sfinstance,
              sandbox = isSandbox)

In [39]:
#validate connection
sf.query("Select Id, Name From Opportunity Limit 1")

OrderedDict([('totalSize', 1),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Opportunity'),
                                          ('url',
                                           '/services/data/v38.0/sobjects/Opportunity/0063i0000032cXEAAY')])),
                            ('Id', '0063i0000032cXEAAY'),
                            ('Name', 'Trout Brewing Company')])])])

<h3 style="color: #5e9ca0;"> At this point assuming you have created a SalesForce Account and established a connection. 
Go ahead and setup a few opportunites at SalesForce.com</h3>

In [40]:
##this definition was created by a developer named Jie Jenn for SOQL calls to SalesForce
## you can watch a how to video by him here: https://www.youtube.com/watch?v=iKaFa3N2Nhw&t=770s
def SalesCall(SalesCall):
    qryResult = sf.query(SalesCall)
    print('Record Count (0)'.format(qryResult['totalSize']))
    isDone = qryResult['done']
    
    if isDone == True:
        df = DataFrame(qryResult['records'])
        
    while isDone != True:
        try:
            if qryResult['done'] != True:
                df = df.append(DataFrame(qryResult['records']));
                qryResult = sf.query_more(qryResult['nextRecordsUrl'],True)
            else:
                    df = df.append(DataFrame(qryResult['records']))
                    isDone = True;
                    print('completed')
                    break;
        except NameError:
            df = DataFrame(qryResult['records'])
            qry = sf.query_more(qryResult['nextRecordsUrl'], True)
                    
    df = df.drop('attributes',axis =1 )
    return df;


## For a list of Fields refer to documentation https://developer.salesforce.com/docs/atlas.en-us.sfFieldRef.meta/sfFieldRef/salesforce_field_reference_Opportunity.htm
##Account name is a custom rule field below
## in order to create it follow these guidelines:https://help.salesforce.com/articleView?id=adding_fields.htm&type=5
df=SalesCall('Select Name,AccountName__c,Amount,ExpectedRevenue,CloseDate FROM Opportunity')
df.head()

Record Count (0)


,Name,AccountName__c,Amount,ExpectedRevenue,CloseDate
0,Trout Brewing Company,Trout Brewing Company,2000000.0,200000.0,2019-08-23
1,Dickenson Mobile Generators,Dickenson plc,15000.0,1500.0,2019-05-25
2,United Oil Office Portable Generators,United Oil & Gas Corp.,125000.0,112500.0,2019-05-13
3,Express Logistics Standby Generator,Express Logistics and Transport,220000.0,220000.0,2019-04-08
4,GenePoint Standby Generator,GenePoint,85000.0,85000.0,2019-05-16


In [41]:
#bring TM1py library components to your Jupyter Notebook
#to learn more about TM1Py check out https://code.cubewise.com/tm1py
from TM1py.Utils import Utils
from TM1py.Objects import Cube, Dimension, Hierarchy, Element, NativeView, Subset
from TM1py.Services import TM1Service
from datetime import timedelta, date
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import build_cellset_from_pandas_dataframe

In [42]:
#bring user name and password for Non-Interactive account from CSV file. 
#This information is in your Welcome Kit and this account will need access to TM1server that you will be using on this tenant
user=creds.iloc[0,0]
password=creds.iloc[0,1]
#also define your tenant
pTenant=creds.iloc[0,2]
#define TM1 server you will be using
pServer=creds.iloc[0,3]
#Tenant URL
base_url = 'https://' + pTenant + '.planning-analytics.ibmcloud.com/tm1/api/' + pServer
#SSLis always true for PA Cloud
ssl=True
#IBMCloud Name Space
namespace='LDAP'

In [43]:
# Time magic with python generator
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


# push data to TM1
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    # ============================


    # create dimension SalesForce Date
    start_date = date(1990, 1, 1)
    end_date = date(2041, 1, 1)
    elements = [Element(str(single_date), 'Numeric') for single_date in daterange(start_date, end_date)]
    hierarchy = Hierarchy('SalesForce Date', 'SalesForce Date', elements)
    dimension = Dimension('SalesForce Date', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

   
    # create dimension Sales Force Opportunities
    #Tickers to create
    opportunities = list(df['Name'])
    elements = [Element(opportunity, 'String') for opportunity in opportunities]
    hierarchy = Hierarchy('SalesForce Opportunities', 'SalesForce Opportunities', elements)
    dimension = Dimension('SalesForce Opportunities', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

    # create dimension Sales Force Accounts
    #Tickers to create
    accounts = list(df['AccountName__c'])
    elements = [Element(account, 'String') for account in accounts]
    hierarchy = Hierarchy('SalesForce Accounts', 'SalesForce Accouns', elements)
    dimension = Dimension('SalesForce Accounts', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)
        
        
    # create dimension SalesForce Measure
    measures = ('Amount','ExpectedRevenue')
    elements = [Element(measure, 'Numeric') for measure in measures]
    hierarchy = Hierarchy('SalesForce Measure', 'SalesForce Measure', elements)
    dimension = Dimension('SalesForce Measure', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

    # create cube TM1py Stock Prices
    #cube = Cube('SalesForce Python', ['SalesForce Opportunities', 'SalesForce Accounts','SalesForce Date', 'SalesForce Measure'])
    cube = Cube('SalesForce Python', ['SalesForce Opportunities', 'SalesForce Accounts','SalesForce Date', 'SalesForce Measure'])
    if not tm1.cubes.exists(cube.name):
        tm1.cubes.create(cube)

In [44]:
df.head()

,Name,AccountName__c,Amount,ExpectedRevenue,CloseDate
0,Trout Brewing Company,Trout Brewing Company,2000000.0,200000.0,2019-08-23
1,Dickenson Mobile Generators,Dickenson plc,15000.0,1500.0,2019-05-25
2,United Oil Office Portable Generators,United Oil & Gas Corp.,125000.0,112500.0,2019-05-13
3,Express Logistics Standby Generator,Express Logistics and Transport,220000.0,220000.0,2019-04-08
4,GenePoint Standby Generator,GenePoint,85000.0,85000.0,2019-05-16


In [45]:
cube = 'SalesForce Python'
pLoad='Amount'
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
        d = {'Name': pd.Series((df['Name'].values)),
             'Account Name': pd.Series((df['AccountName__c'].values)),
             'Close Date': pd.Series((df['CloseDate'].values)),
             'Measure': pLoad,
             'Values': pd.Series(df[pLoad].values)}
        df = pd.DataFrame(d)
        cellset = build_cellset_from_pandas_dataframe(df)
        tm1.cubes.cells.write_values(
            cube_name=cube,
            cellset_as_dict=cellset)

In [ ]:
#optional cleanup cubes
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    cube_names = tm1.cubes.get_all_names()
    for cube_name in cube_names:
        if "SalesForce" in cube_name:
            tm1.cubes.delete(cube_name)

In [ ]:
#optional cleanup dimensions
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    dimension_names = tm1.dimensions.get_all_names()
    for dimension_name in dimension_names:
        if "SalesForce" in dimension_name and "}" not in dimension_name:
            tm1.dimensions.delete(dimension_name)